In [ ]:
import numpy as np


def prime_factorization(n: int):
    """Return the prime factorization of n as a list (using numpy only)."""
    factors = []
    d = 2
    while d * d <= n:
        while n % d == 0:
            factors.append(d)
            n //= d
        d += 1
    if n > 1:
        factors.append(n)
    return factors


def split_into_four_factors(N: int):
    """
    Split integer N into 4 factors that are as close as possible in size.
    Uses numpy only.
    """
    if N <= 0:
        raise ValueError("N must be positive")

    # Step 1: prime factors
    factors = prime_factorization(N)

    # Step 2: initialize four groups
    groups = np.ones(4, dtype=int)

    # Step 3: distribute factors greedily (largest first)
    for f in sorted(factors, reverse=True):
        idx = np.argmin(groups)   # index of smallest product
        groups[idx] *= f

    return tuple(sorted(groups.tolist()))


# =========================
# Example usage
# =========================
if __name__ == "__main__":
    test_numbers = [3600, 2310, 10**6, 9973, 128]

    for n in test_numbers:
        factors = split_into_four_factors(n)
        print(f"{n} -> {factors}, product = {np.prod(factors)}")

3600 -> (6, 6, 10, 10), product = 3600
2310 -> (5, 6, 7, 11), product = 2310
1000000 -> (25, 25, 40, 40), product = 1000000
9973 -> (1, 1, 1, 9973), product = 9973
128 -> (2, 4, 4, 4), product = 128
